Classification multi-perspective approach RoBERTa-large HHD

In [1]:
import ast 
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import numpy as np

from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split



In [2]:

train['soft_labels'] = train['soft_labels'].apply(ast.literal_eval).tolist()
test['soft_labels'] = test['soft_labels'].apply(ast.literal_eval).tolist()
val['soft_labels'] = val['soft_labels'].apply(ast.literal_eval).tolist()

In [3]:

train = Dataset.from_pandas(train)
val = Dataset.from_pandas(val)
test = Dataset.from_pandas(test)

dataset = DatasetDict() 
dataset['train'] = train
dataset['val'] = val
dataset['test'] = test

In [4]:
model_name = 'roberta-large'
model_name_filename = model_name.replace("/", "-")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
def tokenize_func(examples):
    # Tokenize the input text and map the 'soft_labels' column to 'labels'
    tokenized_inputs = tokenizer(examples['Input'], padding='max_length', truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['soft_labels'] 
    return tokenized_inputs

In [7]:
train_tokenized = train.map(tokenize_func, batched = True)
val_tokenized = val.map(tokenize_func, batched = True) 

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [8]:
train_tokenized.set_format('torch', columns =['input_ids', 'attention_mask', 'labels'])
val_tokenized.set_format('torch', columns = ['input_ids', 'attention_mask', 'labels'])

In [9]:
train_tokenized['labels'][0]

tensor([0.9000, 0.0000, 0.0000, 0.0000])

In [10]:
num_labels=len(train_tokenized['labels'][0])

In [11]:
num_labels

4

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
output_dir = f"./multiclassification/{model_name_filename}/results/human"

In [ ]:
#training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
#definition soft loss function

def softmax(logits):
    """
    Compute softmax probabilities from logits.
    
    Parameters:
    - logits: A numpy array of shape (n, num_classes) containing the logits.
    
    Returns:
    - probabilities: A numpy array of shape (n, num_classes) containing the softmax probabilities.
    """
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

def soft_loss_function(true_probabilities, predicted_logits):
    """
    Compute the soft loss function (cross-entropy with soft labels) using PyTorch tensors.
    
    Parameters:
    - true_probabilities: A PyTorch tensor of shape (n, num_classes) containing the true probability distributions.
    - predicted_logits: A PyTorch tensor of shape (n, num_classes) containing the logits from the model.
    
    Returns:
    - loss: The computed soft loss.
    """
    
    predicted_probabilities = torch.nn.functional.softmax(predicted_logits, dim=-1)
    
   
    epsilon = 1e-15
    predicted_probabilities = torch.clamp(predicted_probabilities, epsilon, 1. - epsilon)
    
   
    loss = -torch.sum(true_probabilities * torch.log(predicted_probabilities))
    
    return loss

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        Compute the loss for the model. This uses the soft_loss_function.

        Parameters:
        - model: The model to evaluate.
        - inputs: A dictionary of inputs to the model.
        - return_outputs: Whether to return model outputs along with loss.

        Returns:
        - loss: The computed loss.
        - outputs (optional): The model outputs, if return_outputs is True.
        """
        labels = inputs.pop("labels")
        
      
        outputs = model(**inputs)
        logits = outputs.logits
        
       
        true_probabilities = labels 
        logits = logits 
        
        
        loss = soft_loss_function(true_probabilities, logits)
        
        return (loss, outputs) if return_outputs else loss


In [18]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [21]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir) 

('./multiclassification/roberta-large/results/human/tokenizer_config.json',
 './multiclassification/roberta-large/results/human/special_tokens_map.json',
 './multiclassification/roberta-large/results/human/vocab.json',
 './multiclassification/roberta-large/results/human/merges.txt',
 './multiclassification/roberta-large/results/human/added_tokens.json',
 './multiclassification/roberta-large/results/human/tokenizer.json')

In [22]:
model 

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [23]:
tokenized_test = test.map(tokenize_func, batched=True)
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [24]:
def predict(texts, model, tokenizer, device):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits.cpu().numpy()

In [26]:
all_predictions = []
all_labels = []
batch_size = 8

for i in range(0, len(test), batch_size):
    batch = test[i:i+batch_size]
    batch_texts = batch['Input']
    batch_predictions = predict(batch_texts, model, tokenizer, device)
    all_predictions.extend(batch_predictions)
    all_labels.extend(batch['labels'])

In [27]:
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

In [28]:
from scipy.special import softmax
# Apply softmax to predictions
softmax_predictions = softmax(all_predictions, axis=1)

In [29]:
df_test = test.to_pandas()

In [31]:
df_test['Predicted_scores'] = all_predictions.tolist()
df_test['Predicted_Softmax_scores'] = softmax_predictions.tolist()

In [32]:
df_test['predicted_labels'] = df_test['Predicted_scores'].apply(lambda x: x.index(max(x)))
df_test['predicted_softmax_labels'] = df_test['Predicted_Softmax_scores'].apply(lambda x: x.index(max(x)))

In [33]:
from datasets import Dataset
test = Dataset.from_pandas(df_test) 

In [34]:
test

Dataset({
    features: ['Unnamed: 0', 'docID', 'Query', 'docTitle', 'doc', 'Input', 'labels', 'majority_label', 'label_index', 'soft_labels', 'Predicted_scores', 'Predicted_Softmax_scores', 'predicted_labels', 'predicted_softmax_labels'],
    num_rows: 139
})

In [37]:
#label encoding
label_ecnoding_multip = {'Pro': 0,
'Against': 1,
'Neutral': 2,
'Not-about': 3,}


m_test['true_labels'] = m_test['majority_label'].map(label_ecnoding_multip) 

In [39]:
#calculate classification metrics 
from torch.nn import functional as F

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def calculate_confidences(df, model, tokenizer):
    """
    Calculate confidence scores and update the DataFrame with a new column.
    
    Args:
    - df: DataFrame with input data
    - model: Trained model with a method to get logits
    - tokenizer: Tokenizer to preprocess text
    
    Returns:
    - df: Updated DataFrame with a 'confidence_scores' column
    """
    confidences = []

    model.eval() 

    for index, row in df.iterrows():
        text = row['Input']  
        
       
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
        
        with torch.no_grad():
            logits = model(**inputs).logits
            probabilities = F.softmax(logits, dim=-1)  
            probabilities = probabilities.cpu().numpy().flatten()  
            
           
            max_prob = np.max(probabilities)
            confidences.append(max_prob)
    
   
    df['confidence_scores'] = confidences
    return df


m_test = calculate_confidences(m_test, model, tokenizer)

# Proceed with metrics calculation
y_true = m_test['true_labels']
y_pred = m_test['predicted_labels']
confidence_scores = m_test['confidence_scores']

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy * 100)

# Precision
precision = precision_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Precision:", precision * 100)

# Recall
recall = recall_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Recall:", recall * 100)

# F1 Score
f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("F1 Score:", f1 * 100)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred)
print("Classification Report:")
print(class_report)

# Average Confidence Score
avg_confidence = np.mean(confidence_scores)
print("Average Confidence Score:", avg_confidence * 100)

# Confidence for Correct and Incorrect Predictions
correct_confidence = np.mean([confidence for pred, true, confidence in zip(y_pred, y_true, confidence_scores) if pred == true])
incorrect_confidence = np.mean([confidence for pred, true, confidence in zip(y_pred, y_true, confidence_scores) if pred != true])

print("Average Confidence for Correct Predictions:", correct_confidence * 100)
print("Average Confidence for Incorrect Predictions:", incorrect_confidence * 100)

Accuracy: 60.431654676258994
Precision: 63.553892620752436
Recall: 62.83664127238706
F1 Score: 61.90897381135846
Confusion Matrix:
[[26  0 11  6]
 [ 5 22  0  2]
 [ 9  2 19 13]
 [ 6  0  1 17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.60      0.58        43
           1       0.92      0.76      0.83        29
           2       0.61      0.44      0.51        43
           3       0.45      0.71      0.55        24

    accuracy                           0.60       139
   macro avg       0.64      0.63      0.62       139
weighted avg       0.63      0.60      0.61       139

Average Confidence Score: 48.760855197906494
Average Confidence for Correct Predictions: 51.40508913568088
Average Confidence for Incorrect Predictions: 44.722386327656835


In [41]:
#soft loss function definition and temperature module

def softmax(logits):
    """
    Compute softmax probabilities from logits.
    
    Parameters:
    - logits: A numpy array of shape (n, num_classes) containing the logits.
    
    Returns:
    - probabilities: A numpy array of shape (n, num_classes) containing the softmax probabilities.
    """
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

def soft_loss_function(true_probabilities, predicted_logits):
    """
    Compute the soft loss function (cross-entropy with soft labels) using PyTorch tensors.
    
    Parameters:
    - true_probabilities: A PyTorch tensor of shape (n, num_classes) containing the true probability distributions.
    - predicted_logits: A PyTorch tensor of shape (n, num_classes) containing the logits from the model.
    
    Returns:
    - loss: The computed soft loss.
    """
    
    predicted_probabilities = torch.nn.functional.softmax(predicted_logits, dim=-1)
    

    epsilon = 1e-15
    predicted_probabilities = torch.clamp(predicted_probabilities, epsilon, 1. - epsilon)
    
  
    loss = -torch.sum(true_probabilities * torch.log(predicted_probabilities))
    
    return loss

class TemperatureScalingCalibration(nn.Module):
    def __init__(self, model_path: str, tokenizer, device: torch.device):
        super().__init__()
        self.model_path = model_path
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = tokenizer
        self.device = device

    
        self.temperature = nn.Parameter(torch.ones(1)) 
        
        self.model.to(self.device)
        self.temperature.to(self.device)

    def forward(self, input_ids, attention_mask):
        """Forward method that returns softmax-ed confidence scores."""
        logits = self.forward_logit(input_ids, attention_mask)
        scaled_logits = logits / self.temperature
        scores = nn.functional.softmax(scaled_logits, dim=-1)
        return scores

    def forward_logit(self, input_ids, attention_mask):
        """Forward method that returns logits, to be used with cross-entropy loss."""
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        ).logits
        return outputs

    def fit(self, dataset_tokenized, n_epochs: int = 3, batch_size: int = 64, lr: float = 0.01):
        """Fits the temperature scaling parameter."""
        data_collator = DataCollatorWithPadding(self.tokenizer, padding=True)
        data_loader = DataLoader(dataset_tokenized, collate_fn=data_collator, batch_size=batch_size)

        self.freeze_base_model()
      
  
        optimizer = optim.SGD(self.parameters(), lr=lr)

      
        self.train()

        for epoch in trange(n_epochs):
            epoch_loss = 0.0
            for examples in data_loader:
                
                input_ids = examples['input_ids'].to(self.device)
                attention_mask = examples['attention_mask'].to(self.device)
                soft_labels = examples['labels'].to(self.device)  
                
                
                self.zero_grad()
                logits = self.forward_logit(input_ids, attention_mask)
                
                
                scaled_logits = logits / self.temperature
                
                
                loss = soft_loss_function(soft_labels, scaled_logits)
                loss.backward()
                optimizer.step()

                
                epoch_loss += loss.item() * input_ids.size(0)

        
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss / len(dataset_tokenized)}")

        return self

    def freeze_base_model(self):
        """Remember to freeze base model's parameters when training temperature scaler."""
        self.model.eval()
        for parameter in self.model.parameters():
            parameter.requires_grad = False

        return self


In [43]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir) 

('./multiclassification/roberta-large/results/human/tokenizer_config.json',
 './multiclassification/roberta-large/results/human/special_tokens_map.json',
 './multiclassification/roberta-large/results/human/vocab.json',
 './multiclassification/roberta-large/results/human/merges.txt',
 './multiclassification/roberta-large/results/human/added_tokens.json',
 './multiclassification/roberta-large/results/human/tokenizer.json')

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
calibration_module = TemperatureScalingCalibration(model_path=output_dir, tokenizer=tokenizer, device=device)
calibration_module.to(device) 

TemperatureScalingCalibration(
  (model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(

In [45]:
def tokenize_func(examples):
    # Tokenize the input text and map the 'soft_labels' column to 'labels'
    tokenized_inputs = tokenizer(examples['Input'], padding='max_length', truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['soft_labels'] 
    return tokenized_inputs

In [46]:
val_tokenized_cal = val.map(tokenize_func, batched = True) 

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [47]:
val_tokenized_cal.set_format('torch', columns = ['input_ids', 'attention_mask', 'labels'])

In [48]:
calibration_module.fit(val_tokenized_cal,n_epochs=6, batch_size=64, lr=0.01) 

 17%|█▋        | 1/6 [00:04<00:21,  4.33s/it]

Epoch 1/6, Loss: 72.29785212509924


 33%|███▎      | 2/6 [00:08<00:17,  4.32s/it]

Epoch 2/6, Loss: 72.93254679398571


 50%|█████     | 3/6 [00:12<00:12,  4.33s/it]

...

 83%|████████▎ | 5/6 [00:21<00:04,  4.35s/it]

Epoch 5/6, Loss: 71.6213482575451


100%|██████████| 6/6 [00:26<00:00,  4.34s/it]

Epoch 6/6, Loss: 71.56138588198655


TemperatureScalingCalibration(
  (model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(

In [49]:
calibration_module.eval()  

TemperatureScalingCalibration(
  (model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(

In [50]:
def predict_cal(texts, model, tokenizer, device):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items() if k in ['input_ids', 'attention_mask']} 
    with torch.no_grad():
        logits = model(**inputs)  
    return logits.cpu().numpy()

In [52]:
all_predictions = []
all_labels = []
batch_size = 8  

for i in range(0, len(test), batch_size):
    batch = test[i:i+batch_size]
    batch_texts = batch['Input']
    batch_predictions = predict_cal(batch_texts, calibration_module, tokenizer, device)
    all_predictions.extend(batch_predictions)
    all_labels.extend(batch['labels']) 

In [53]:
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

In [54]:
from scipy.special import softmax
# Apply softmax to predictions
softmax_predictions = softmax(all_predictions, axis=1)

In [55]:
df_test_cal = test.to_pandas()

In [56]:
df_test_cal['Predicted_scores'] = all_predictions.tolist()
df_test_cal['Predicted_Softmax_scores'] = softmax_predictions.tolist()

In [57]:
df_test_cal['predicted_labels'] = df_test['Predicted_scores'].apply(lambda x: x.index(max(x)))
df_test_cal['predicted_softmax_labels'] = df_test['Predicted_Softmax_scores'].apply(lambda x: x.index(max(x)))

In [58]:
from datasets import Dataset
test_cal = Dataset.from_pandas(df_test_cal) 

In [61]:
label_ecnoding_multip = {'Pro': 0,
'Against': 1,
'Neutral': 2,
'Not-about': 3,} 


m_test_cal['true_labels'] = m_test_cal['majority_label'].map(label_ecnoding_multip)  

In [62]:
from torch.nn import functional as F

def calculate_confidences_cal(df, model, tokenizer):
    """
    Calculate confidence scores and update the DataFrame with a new column.
    
    Args:
    - df: DataFrame with input data
    - model: Trained model with a method to get logits
    - tokenizer: Tokenizer to preprocess text
    
    Returns:
    - df: Updated DataFrame with a 'confidence_scores' column
    """
    confidences = []

    model.eval()  

    for index, row in df.iterrows():
        text = row['Input']  
        
     
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
        
        with torch.no_grad():
            logits = model(**inputs).logits
            probabilities = F.softmax(logits, dim=-1)  
            probabilities = probabilities.cpu().numpy().flatten()  
            
           
            max_prob = np.max(probabilities)
            confidences.append(max_prob)
    
  
    df['confidence_scores'] = confidences
    return df


m_test_cal = calculate_confidences_cal(m_test_cal, model, tokenizer)


y_true = m_test_cal['true_labels']
y_pred = m_test_cal['predicted_labels']
confidence_scores = m_test_cal['confidence_scores']

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy * 100)

# Precision
precision = precision_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Precision:", precision * 100)

# Recall
recall = recall_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Recall:", recall * 100)

# F1 Score
f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("F1 Score:", f1 * 100)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred)
print("Classification Report:")
print(class_report)

# Average Confidence Score
avg_confidence = np.mean(confidence_scores)
print("Average Confidence Score:", avg_confidence * 100)

# Confidence for Correct and Incorrect Predictions
correct_confidence = np.mean([confidence for pred, true, confidence in zip(y_pred, y_true, confidence_scores) if pred == true])
incorrect_confidence = np.mean([confidence for pred, true, confidence in zip(y_pred, y_true, confidence_scores) if pred != true])

print("Average Confidence for Correct Predictions:", correct_confidence * 100)
print("Average Confidence for Incorrect Predictions:", incorrect_confidence * 100)

Accuracy: 60.431654676258994
Precision: 63.553892620752436
Recall: 62.83664127238706
F1 Score: 61.90897381135846
Confusion Matrix:
[[26  0 11  6]
 [ 5 22  0  2]
 [ 9  2 19 13]
 [ 6  0  1 17]]
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.60      0.58        43
           1       0.92      0.76      0.83        29
           2       0.61      0.44      0.51        43
           3       0.45      0.71      0.55        24

    accuracy                           0.60       139
   macro avg       0.64      0.63      0.62       139
weighted avg       0.63      0.60      0.61       139

Average Confidence Score: 48.760855197906494
Average Confidence for Correct Predictions: 51.40508913568088
Average Confidence for Incorrect Predictions: 44.722386327656835
